# Name Change
## Aim

The aim of this pipeline is to change the header of gene expression file based on a reference file that are made externally. 

An example of the pipeline running is shown below

In [4]:
SOS run /Users/haosun/Documents/WG_Reasearch_Assisstant/GIT/freshcopy/neuro-twas/Workflow/SOS_ROSMAP_gene_exp_processing.ipynb \
    --ref "/Users/haosun/Documents/WG_Reasearch_Assisstant/Rosmap/Data/new_gene_keys_plink.txt" \
    --name_col 2 \
    --real_name_col 1 \
    --gene_exp "/Users/haosun/Documents/WG_Reasearch_Assisstant/Rosmap/Data/gene_exp/geneCountsResidualsPlusBaselineAgeGenderAdj.txt" \
    --start_at 2 \
    --output "/Users/haosun/Documents/WG_Reasearch_Assisstant/Rosmap/Data/gene_exp/" 

INFO: Running Processing_1: change the name of gene_exp
INFO: Processing_1 (index=0) is ignored due to saved signature
INFO: Processing_1 output:   /Users/haosun/Documents/WG_Reasearch_Assisstant/Rosmap/Data/gene_exp/geneCountsResidualsPlusBaselineAgeGenderAdj_rename.txt /Users/haosun/Documents/WG_Reasearch_Assisstant/Rosmap/Data/gene_exp/geneCountsResidualsPlusBaselineAgeGenderAdj_discardID.txt
INFO: Running Processing_2: add position to gene_ID based on ensembl
INFO: Processing_2 is completed.
INFO: Processing_2 output:   /Users/haosun/Documents/WG_Reasearch_Assisstant/Rosmap/Data/gene_exp/geneCountsResidualsPlusBaselineAgeGenderAdj_rename_anno.txt
INFO: Workflow Processing (ID=7996406c03692b3d) is executed successfully with 1 completed step and 1 ignored step.



/Users/haosun/Documents/WG_Reasearch_Assisstant/GIT/twas-dev/WIP/pipeline_prototype



In [73]:
[global]
# This pipeline changes the patient names of a gene_exp file based on a index reference file 
parameter: ref = "/Users/haosun/Documents/WG_Reasearch_Assisstant/Rosmap/Data/new_gene_keys_plink.txt"
# A table contaiing the patient ID in the gene_exp file and the patient ID in the plink file
parameter: name_col = 2
# Column position that specify the header of gene_exp in the index
parameter: real_name_col = 1
# Column position that specify the plink patient ID in the index
parameter: gene_exp = "/Users/haosun/Documents/WG_Reasearch_Assisstant/Rosmap/Data/gene_exp/geneCountsResidualsPlusBaselineAgeGenderAdj.txt"
# the patient ID to be changed
parameter: start_at = 2
# the column where the patient ID start

parameter: output = "/Users/haosun/Documents/WG_Reasearch_Assisstant/Rosmap/Data/"
# where to store the output file

# change the name of gene_exp
[Processing_1]
input: ref, gene_exp
output: f'{output}/{_input[1]:bn}_rename.txt',
        f'{output}/{_input[1]:bn}_discardID.txt'

R:  expand= "$[ ]" , stderr = f'{_output[1]:n}.stderr', stdout = f'{_output[1]:n}.stdout'
    library(dplyr)
    gene_exp = readr::read_delim("$[_input[1]]",delim = " ")
    index = readr::read_delim("$[_input[0]]",delim = "\t")
    name = data.frame(Name = index%>%pull(colnames(index)[$[name_col]]),Real_name = index%>%pull(colnames(index)[$[real_name_col]]))
    df = gene_exp[,$[start_at]:ncol(gene_exp)]
    pos = match(colnames(df), name$Name%>%as.character())%>%na.omit()
    df = df %>%
      select_at(as.character(name$Name[pos]), 
                ~as.character((name$Real_name[pos])))
    final = cbind(gene_exp[,1:$[start_at]-1],df)
    final%>%readr::write_tsv( path = "$[_output[0]]", na = "NA", append = FALSE, col_names = TRUE, quote_escape = "double")
    name$Name[attr(pos,"na.action")]%>%cat(file = "$[_output[1]]", sep = " ")

# add position to gene_ID based on ensembl 
[Processing_2]
input: group_by = 2
output: f'{output}/{_input[0]:bn}_anno.txt'

R:  expand= "$[ ]", stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout'
    library("biomaRt")
    library(dplyr)
    gene_exp = readr::read_delim("$[_input[0]]",delim = "\t")
    ensembl = useDataset("hsapiens_gene_ensembl",mart=useMart("ENSEMBL_MART_ENSEMBL"))
    ensembl_df <- getBM(attributes=c("ensembl_gene_id","chromosome_name", "start_position", "end_position"),mart=ensembl)
    my_genes = gene_exp$gene_ID
    my_genes_ann = ensembl_df[match(my_genes, ensembl_df$ensembl_gene_id),]%>%dplyr::select("gene_ID" = ensembl_gene_id, "chr" = chromosome_name, start_position, end_position)
    final = inner_join(my_genes_ann, gene_exp, "gene_ID")
    final%>%readr::write_tsv( path = "$[_output]", na = "NA", append = FALSE, col_names = TRUE, quote_escape = "double")

INFO: Running default :

HINT: Pulling docker image gaow/twas
/usr/local/bin/gcta64


INFO: Workflow default (ID=725ab0a3d301da69) is executed successfully with 1 completed step.

PLINK v1.90b6.18 64-bit (16 Jun 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3

  plink <input flag(s)...> [command flag(s)...] [other flag(s)...]
  plink --help [flag name(s)...]

Commands include --make-bed, --recode, --flip-scan, --merge-list,
--write-snplist, --list-duplicate-vars, --freqx, --missing, --test-mishap,
--hardy, --mendel, --ibc, --impute-sex, --indep-pairphase, --r2, --show-tags,
--blocks, --distance, --genome, --homozyg, --make-rel, --make-grm-gz,
--rel-cutoff, --cluster, --pca, --neighbour, --ibs-test, --regress-distance,
--model, --bd, --gxe, --logistic, --dosage, --lasso, --test-missing,
--make-perm-pheno, --tdt, --qfam, --annotate, --clump, --gene-report,
--meta-analysis, --epistasis, --fast-epistasis, and --score.

"plink --help | more" describes all functions (warning: long).
/usr/local/bin/plink



Warning message in install.packages("data.table"):
“installation of package ‘data.table’ had non-zero exit status”
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



{'chr': 'dir',
 '1': '/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/Project/test/LDREF/1000G.EUR.1.bed',
 '2': '/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/Project/test/LDREF/1000G.EUR.2.bed',
 '3': '/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/Project/test/LDREF/1000G.EUR.3.bed',
 '4': '/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/Project/test/LDREF/1000G.EUR.4.bed',
 '5': '/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/Project/test/LDREF/1000G.EUR.5.bed',
 '6': '/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/Project/test/LDREF/1000G.EUR.6.bed',
 '7': '/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/Project/test/LDREF/1000G.EUR.7.bed',
 '8': '/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/Project/test/LDREF/1000G.EUR.8.bed',
 '9': '/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/Project/test/LDREF/1000G.EUR.9.bed',
 '10': '/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/Project/test/LDREF/1000G.EUR.10

{'chr': 'dir',
 '1': '/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/Project/test/LDREF/1000G.EUR.1.bed',
 '2': '/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/Project/test/LDREF/1000G.EUR.2.bed',
 '3': '/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/Project/test/LDREF/1000G.EUR.3.bed',
 '4': '/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/Project/test/LDREF/1000G.EUR.4.bed',
 '5': '/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/Project/test/LDREF/1000G.EUR.5.bed',
 '6': '/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/Project/test/LDREF/1000G.EUR.6.bed',
 '7': '/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/Project/test/LDREF/1000G.EUR.7.bed',
 '8': '/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/Project/test/LDREF/1000G.EUR.8.bed',
 '9': '/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/Project/test/LDREF/1000G.EUR.9.bed',
 '10': '/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/Project/test/LDREF/1000G.EUR.10

'/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/Project/test/LDREF/1000G.EUR.3.bed'

False

False

/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/Project/Updated/Working/cache/gene_exp_file.ENSG00000183308.6


7

'7\n6\n2\n3\n5\n1\n2\n4\n3\n1\n7\n1\n6\n1\n4\n6\n2\n5\n2\n7\n6\n2\n6\n7\n1\n6\n4\n7\n'